# Packages  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

C:\Users\leona\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Preparing Data

For the sake of simplicity we assume the data is an incremental run off triangle with 5 accident yea and 5 development year as well

In [2]:
tr_inc = pd.DataFrame([[10, 20, 15, 35, 10],
                        [50, 20, 80, 10], 
                        [10, 30, 20], 
                        [10, 20], 
                        [10]], dtype = float)

In [3]:
tr_inc

,0,1,2,3,4
0,10.0,20.0,15.0,35.0,10.0
1,50.0,20.0,80.0,10.0,NaN
2,10.0,30.0,20.0,NaN,NaN
3,10.0,20.0,NaN,NaN,NaN
4,10.0,NaN,NaN,NaN,NaN


For Chain Ladder Method we need to change the incremental run off triangle to cumulative run-off triangle

In [4]:
tr_cum = tr_inc.apply(np.cumsum, axis = 1)
tr_cum

,0,1,2,3,4
0,10.0,30.0,45.0,80.0,90.0
1,50.0,70.0,150.0,160.0,NaN
2,10.0,40.0,60.0,NaN,NaN
3,10.0,30.0,NaN,NaN,NaN
4,10.0,NaN,NaN,NaN,NaN


# Chain Ladder Method  


## Development Factor


\begin{equation} \labels{dev}
    f_k = \frac{\sum_{j=1}^{I-k} C_{i+1,j}}{\sum_{j=1}^{I-k} C_{ij}}
\end{equation}

In [58]:
def dev_factor(i):
    return np.sum(tr_cum[i+1][:-(i+1)])/np.sum(tr_cum[i][:-(i+1)])

In [59]:
fk = pd.DataFrame([dev_factor(i) for i in range(tr_cum.shape[0]-1)], columns = ['fk'])
fk

,fk
0,2.125000
1,1.821429
2,1.230769
3,1.125000


In [116]:
def pred_cum(i):
    return tr_cum[i-1][j]*dev_factor(i-1)

In [122]:
full = tr_cum.copy()
for i in range(full.shape[0]):
    for j in range(full.shape[1]):
        if np.isnan(tr_cum[i][j]) == True :  
            full[i][j] = predict[i-1][j]*dev_factor(i-1)
full

,0,1,2,3,4
0,10.0,30.00,45.000000,80.000000,90.000000
1,50.0,70.00,150.000000,160.000000,180.000000
2,10.0,40.00,60.000000,73.846154,83.076923
3,10.0,30.00,54.642857,67.252747,75.659341
4,10.0,21.25,38.705357,47.637363,53.592033


In [123]:
predict = full.copy()
for i in range(predict.shape[0]):
    for j in range(predict.shape[1]):
        if np.isnan(tr_cum[i][j]) == False :  
            predict[i][j] = np.nan
predict

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,180.000000
2,NaN,NaN,NaN,73.846154,83.076923
3,NaN,NaN,54.642857,67.252747,75.659341
4,NaN,21.25,38.705357,47.637363,53.592033
